In [ ]:
# Anipose visualization

In [2]:
from mayavi import mlab
mlab.options.offscreen = True

import numpy as np
from glob import glob
import pandas as pd
import os
import os.path
import toml
import cv2
import skvideo.io
from tqdm import tqdm, trange
import sys
from collections import defaultdict
from matplotlib.pyplot import get_cmap

# from .common import make_process_fun, get_nframes, get_video_name, get_video_params, get_data_length, natural_keys

In [3]:
def connect(points, bps, bp_dict, color):
    ixs = [bp_dict[bp] for bp in bps]
    return mlab.plot3d(points[ixs, 0], points[ixs, 1], points[ixs, 2],
                       np.ones(len(ixs)), reset_zoom=False,
                       color=color, tube_radius=None, line_width=10)

def connect_all(points, scheme, bp_dict, cmap):
    lines = []
    for i, bps in enumerate(scheme):
        line = connect(points, bps, bp_dict, color=cmap(i)[:3])
        lines.append(line)
    return lines

def update_line(line, points, bps, bp_dict):
    ixs = [bp_dict[bp] for bp in bps]
    # ixs = [bodyparts.index(bp) for bp in bps]
    new = np.vstack([points[ixs, 0], points[ixs, 1], points[ixs, 2]]).T
    line.mlab_source.points = new

def update_all_lines(lines, points, scheme, bp_dict):
    for line, bps in zip(lines, scheme):
        update_line(line, points, bps, bp_dict)

In [12]:
config_file = '/home/yiting/Documents/Anipose_projects/config_2cam.toml'
p3d_file = '/home/yiting/Documents/Anipose_projects/Anipose_240624/p3d.csv'
output_file = '/home/yiting/Documents/Anipose_projects/Anipose_240624/p3d.mp4'

In [13]:
# Load the TOML file
config = toml.load(config_file)

# Access the loaded data
print(config)


{'project': '2cam_test', 'path': '/home/yiting/Documents/Anipose_projects', 'model_folder': '/home/tuthill/lili/flypose-test-Lili-2018-12-10', 'nesting': 20, 'video_extension': 'mp4', 'calibration': {'board_type': 'checkerboard', 'board_size': [4, 3], 'board_marker_bits': 6, 'board_marker_dict_number': 250, 'board_marker_length': 9, 'board_square_side_length': 15, 'animal_calibration': False, 'calibration_init': 'calibration_2cam.toml'}, 'filter': {'enabled': True, 'medfilt': 13, 'offset_threshold': 25, 'score_threshold': 0.5, 'spline': True}, 'labeling': {'scheme': [['index_PIP', 'index_DIP', 'index_Tip'], ['middle_PIP', 'middle_DIP', 'middle_Tip'], ['ring_PIP', 'ring_DIP', 'ring_Tip'], ['small_PIP', 'small_DIP', 'small_Tip']]}, 'triangulation': {'cam_regex': 'cam([A-Z])', 'optim': True, 'axes': [['x', 'L1A', 'L3A'], ['z', 'L1B', 'L1A']], 'reference_point': 'L1A', 'cam_align': 'C'}, 'angles': {'L1_CF': ['L1A', 'L1B', 'L1C'], 'L1_FTi': ['L1B', 'L1C', 'L1D'], 'L1_TiTa': ['L1C', 'L1D', '

In [6]:
def visualize_labels(config, labels_fname, outname, fps=100):

    try:
        scheme = config['labeling']['scheme']
    except KeyError:
        scheme = []

    data = pd.read_csv(labels_fname)
    cols = [x for x in data.columns if '_error' in x]

    if len(scheme) == 0:
        bodyparts = [c.replace('_error', '') for c in cols]
    else:
        bodyparts = sorted(set([x for dx in scheme for x in dx]))

    bp_dict = dict(zip(bodyparts, range(len(bodyparts))))

    all_points = np.array([np.array(data.loc[:, (bp+'_x', bp+'_y', bp+'_z')])
                           for bp in bodyparts], dtype='float64')

    all_errors = np.array([np.array(data.loc[:, bp+'_error'])
                           for bp in bodyparts], dtype='float64')

    all_scores = np.array([np.array(data.loc[:, bp+'_score'])
                           for bp in bodyparts], dtype='float64')

    all_ncams = np.array([np.array(data.loc[:, bp+'_ncams'])
                          for bp in bodyparts], dtype='float64')


    if config['triangulation']['optim']:
        all_errors[np.isnan(all_errors)] = 0
    else:
        all_errors[np.isnan(all_errors)] = 10000
    good = (all_errors < 100)
    all_points[~good] = np.nan

    not_enough_points = np.mean(all_ncams >= 2, axis=1) < 0.2
    all_points[not_enough_points] = np.nan

    all_points_flat = all_points.reshape(-1, 3)
    check = ~np.isnan(all_points_flat[:, 0])

    if np.sum(check) < 10:
        print('too few points to plot, skipping...')
        return

    low, high = np.percentile(all_points_flat[check], [5, 95], axis=0)

    nparts = len(bodyparts)
    framedict = dict(zip(data['fnum'], data.index))

    writer = skvideo.io.FFmpegWriter(outname, inputdict={
        # '-hwaccel': 'auto',
        '-framerate': str(fps),
    }, outputdict={
        '-vcodec': 'h264', '-qp': '28', '-pix_fmt': 'yuv420p'
    })

    cmap = get_cmap('tab10')


    points = np.copy(all_points[:, 20])
    points[0] = low
    points[1] = high

    s = np.arange(points.shape[0])
    good = ~np.isnan(points[:, 0])

    fig = mlab.figure(bgcolor=(1,1,1), size=(500,500))
    fig.scene.anti_aliasing_frames = 2

    low, high = np.percentile(points[good, 0], [10,90])
    scale_factor = (high - low) / 12.0

    mlab.clf()
    pts = mlab.points3d(points[:, 0], points[:, 1], points[:, 2], s,
                        color=(0.8, 0.8, 0.8),
                        scale_mode='none', scale_factor=scale_factor)
    lines = connect_all(points, scheme, bp_dict, cmap)
    mlab.orientation_axes()

    view = list(mlab.view())

    mlab.view(focalpoint='auto', distance='auto')

    for framenum in trange(data.shape[0], ncols=70):
        fig.scene.disable_render = True

        if framenum in framedict:
            points = all_points[:, framenum]
        else:
            points = np.ones((nparts, 3))*np.nan

        s = np.arange(points.shape[0])
        good = ~np.isnan(points[:, 0])

        new = np.vstack([points[:, 0], points[:, 1], points[:, 2]]).T
        pts.mlab_source.points = new
        update_all_lines(lines, points, scheme, bp_dict)

        fig.scene.disable_render = False

        img = mlab.screenshot()

        mlab.view(*view, reset_roll=False)

        writer.writeFrame(img)

    mlab.close(all=True)
    writer.close()

In [14]:
# 3d labels
visualize_labels(config, p3d_file, output_file, fps=100)

100%|█████████████████████████████| 1425/1425 [00:17<00:00, 83.31it/s]
